In [1]:
!pip install -q catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from catboost import Pool
from catboost import CatBoostRegressor

import pickle

In [16]:
DATA_PATH = "/content/drive/MyDrive/itmo/data_nlp/matching_results.csv"
df = pd.read_csv(DATA_PATH)

df_test_example = df.drop(["Matching_Skills"], axis=1).iloc[[3]].copy()
df_test_example.head()

,Vacancy,Employer,Vac_salary,Vac_salary_from,Vac_salary_to,Vac_exp,Vac_schedule,Vac_description,Vac_prof_roles,Vac_specializations,...,CV_keys,CV_name,CV_specializations,CV_profar_names,CV_exp,CV_schedule,CV_employment,CV_area_name,CV_salary_from,CV_salary_to
3,Ведущий SMM специалист,Импорт Хоум,True,60000.0,80000.0,От 1 года до 3 лет,Полный день,В данный момент мы ищем в нашу команду самого ...,"['SMM-менеджер', 'контент-менеджер']","['Управление маркетингом', ""'PR"", 'Маркетингов...",...,"['Web-дизайн', ""'Полиграфический дизайн"", ""'We...",Web-designer (Веб-дизайнер UI/UX) в веб-студию,"'Начальный уровень, Мало опыта', 'Арт-директор...","'Искусство, развлечения, масс-медиа', 'Маркети...",Нет опыта,Полный день,Полная занятость,Москва,55000.0,80000.0


In [18]:
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    df['Vac_salary_from'] = pd.to_numeric(df['Vac_salary_from'].apply(lambda x: x if x != "Не указана" else np.NaN))
    df['Vac_salary_to'] = pd.to_numeric(df['Vac_salary_to'].apply(lambda x: x if x != "Не указана" else np.NaN))
    return df


def load_model(model_weights_path: str) -> CatBoostRegressor:
    model = CatBoostRegressor()
    model.load_model(model_weights_path)
    return model


def inference(df: pd.DataFrame, model_weights: str) -> float:
    df_prep = preprocess_data(df)
    model = load_model(model_weights)

    pred = model.predict(df_prep)[0]
    pred_clipped = np.clip(pred, 0, 1)
    return pred_clipped.item()

In [20]:
inference(df_test_example, "/content/drive/MyDrive/itmo/catboost_model")

0.15584883653373702